# Data preparation notebook

<ol>
<li> uses the half hourly data </li> 
<li> sorts each block of the  dataset by time and fills forward the hourly weather data </li>
<li> converts time stamp columns minute of day,  day of week, month etc </li>      
<li> converts categorical columns to ints  (not one hot encoded ).  The mapping is persisted in nested dictionary called `cat_map.pkl`</li>   
<li> each block file after being processes with the above feature engineering is appended to a bcolz array </li>     
</ol>


In [11]:
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
import bcolz
import shutil
import pickle
import dask
import modin.pandas as pd

In [12]:
label_col = 'energy(kWh/hh)'
categorical_cols = ['LCLid', 'stdorToU', 'Acorn_grouped', 'Acorn', 'summary', 'h_summary', 'icon', 'h_icon', 'h_precipType']
numeric_cols = ['h_visibility','h_windBearing','h_temperature','h_dewPoint','h_pressure','h_apparentTemperature'
,'h_windSpeed','h_humidity','month_x','dayofweek','isHoliday','halfhourofday','temperatureMax'
,'windBearing','dewPoint','cloudCover','windSpeed','pressure','apparentTemperatureHigh','visibility','humidity','apparentTemperatureLow'
,'apparentTemperatureMax','uvIndex','temperatureLow','temperatureMin','temperatureHigh','apparentTemperatureMin'
,'moonPhase','temperatureMinTime_mod','temperatureMaxTime_mod','apparentTemperatureMinTime_mod','apparentTemperatureMaxTime_mod'
,'temperatureHighTime_mod','temperatureLowTime_mod','apparentTemperatureHighTime_mod','apparentTemperatureLowTime_mod'
,'sunsetTime_mod','sunriseTime_mod','uvIndexTime_mod']
allcols = np.concatenate([categorical_cols ,  numeric_cols ]).tolist()
allcols.append(label_col)
cat_map= dict()
for c in categorical_cols:
    cat_map[c]=dict()

In [13]:
def getvalue_for_column(colname,value):
    global cat_map
    if not value in cat_map[colname].keys():
        td = cat_map[colname]
        cnt = len(td.values())
        td[value]=cnt+1
        cat_map[colname]=td
    return cat_map[colname][value]

In [14]:
def half_hour_of_day(datestring, formatstr='%Y-%m-%d %H:%M:%S.%f', stripChars=6):
    if stripChars>0:
        tmp = datetime.strptime(datestring[:-stripChars], formatstr)
    else:
        tmp = datetime.strptime(datestring, formatstr)
    return 2*(tmp.hour + (tmp.minute /60.0))

In [15]:
def minute_of_day(datestring, formatstr='%Y-%m-%d %H:%M:%S.%f', stripChars=6):
    if type(datestring)==float:
        return np.nan
    if stripChars>0:
        tmp = datetime.strptime(datestring[:-stripChars], formatstr)
    else:
        tmp = datetime.strptime(datestring, formatstr)
    return ((tmp.hour*60) + tmp.minute )

In [16]:
def get_ts_int(datestring, formatstr='%Y-%m-%d %H:%M:%S.%f', stripChars=6):
    if type(datestring)==float:
        return np.nan
    if stripChars>0:
        tmp = datetime.strptime(datestring[:-stripChars], formatstr)
    else:
        tmp = datetime.strptime(datestring, formatstr)
    start = datetime(2000,1, 1 )
    delta = tmp-start
    return delta.days*24*3600 + delta.seconds

In [17]:
def convert_time_cols(data_frame, column_name):
    data_frame[column_name+ "_mod"] = data_frame[column_name].apply(lambda x: minute_of_day(x, formatstr='%Y-%m-%d %H:%M:%S', stripChars=0) )

In [18]:
DATA_ROOT = "data"
BLOCK_PATH = os.path.join(DATA_ROOT,'halfhourly_dataset') 
BLOCKS = os.listdir(BLOCK_PATH)

In [19]:
def persist_bcolz(chunk_number, data, processed_data_dir):
    global da
    if chunk_number == 0:
        if os.path.isdir(processed_data_dir):
            shutil.rmtree(processed_data_dir)
        da = bcolz.carray(data, rootdir=processed_data_dir)
        #da.flush()
    else: 
        #da = bcolz.open(rootdir=processed_data_dir, mode='w')
        da.append(data)
        #da.flush()    

In [20]:
hourly_weather = pd.read_csv(os.path.join(DATA_ROOT, "weather_hourly_darksky.csv"))
daily_weather  = pd.read_csv(os.path.join(DATA_ROOT, "weather_daily_darksky.csv"))
house          = pd.read_csv(os.path.join(DATA_ROOT, "informations_households.csv"))
holidays       = pd.read_csv(os.path.join(DATA_ROOT, "uk_bank_holidays.csv"))

In [21]:
hourly_weather.columns = ["h_"+ c for c in hourly_weather.columns]
hourly_weather.rename(columns={"h_time":"time"}, inplace=True)
hols = holidays['Bank holidays'].values

In [22]:
daily_weather['month']=[ int(v[5:7]) for v in daily_weather['time'].values]
for c in ['temperatureMinTime', 'temperatureMaxTime', 'apparentTemperatureMinTime','apparentTemperatureMaxTime','temperatureHighTime','temperatureLowTime','apparentTemperatureHighTime','apparentTemperatureLowTime','sunsetTime','sunriseTime','uvIndexTime']:
    convert_time_cols(daily_weather, c)
daily_weather['date']=[ v[:10] for v in daily_weather['temperatureMinTime'].values]  
daily_weather.fillna(method='ffill', inplace=True)    

In [35]:
## per block operations
def feature_eng(df):
    df['time']=  df['tstp'].str.slice(0,19)
    df = pd.merge(df, house, on='LCLid', how='left')
    df = pd.merge(df, hourly_weather, on='time',  how='left')
    df['month']=[ int(v[5:7]) for v in df['tstp'].values]
    df['dayofweek']=[datetime.strptime(v[:-6], '%Y-%m-%d %H:%M:%S.%f').weekday()  for v in df['tstp'].values]
    df['isHoliday']=[v[:10] in hols  for v in df['tstp'].values]
    df['halfhourofday'] = df.tstp.apply(lambda x: half_hour_of_day(x) )
    df['date']=[ v[:10] for v in df['tstp'].values] 
    df['ts_int'] = df.tstp.apply(lambda x: get_ts_int(x))
    df = pd.merge(df, daily_weather, on='date', how='left')
    df.sort_values(by='ts_int', ascending=True, inplace=True)
    hcols =['h_visibility', 'h_windBearing', 'h_temperature','h_dewPoint', 'h_pressure', 'h_apparentTemperature', 'h_windSpeed',
       'h_precipType', 'h_icon', 'h_humidity', 'h_summary']
    df[hcols]= df[hcols].ffill()
    df[label_col] = pd.to_numeric(df[label_col], errors='coerce')
    for c in categorical_cols:  # convert cat columns to ints
        df[c] = df[c].apply(lambda x: getvalue_for_column(c,x))
    return df[allcols]

In [36]:
for i, block in enumerate (BLOCKS):
    print ("starting  block ", i)
    ddf = pd.read_csv(os.path.join(BLOCK_PATH,block))
    df = feature_eng(ddf)
    persist_bcolz(i,df[allcols].values.astype(np.float32), 'data/processed_main')
    del ddf
    del df
    print ("completed block ", i)    
da.flush()
# save the string mapping dictionary    
with open("data/cat_map.pkl", "wb") as output_file:
       pickle.dump(cat_map, output_file)

starting  block  0


To request implementation, send an email to feature_requests@modin.org.
  (0, <function PandasQueryCompiler.insert.<locals>. ... 4a6d531b88243')
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


CancelledError: 

In [40]:
da = bcolz.open(rootdir='data/processed_main', mode='r')

In [53]:
len(da)

171240269

In [27]:
i=0
block = 'block_83.csv'
ddf = pd.read_csv(os.path.join(BLOCK_PATH,block))
print(ddf.head())

       LCLid                         tstp energy(kWh/hh)
0  MAC000009  2012-09-20 10:00:00.0000000         0.119 
1  MAC000009  2012-09-20 10:30:00.0000000         0.168 
2  MAC000009  2012-09-20 11:00:00.0000000         0.293 
3  MAC000009  2012-09-20 11:30:00.0000000         0.747 
4  MAC000009  2012-09-20 12:00:00.0000000         0.186 


In [34]:
ddf['time']=  ddf['tstp'].str.slice(0,19)
print(ddf.head())

       LCLid                         tstp energy(kWh/hh)                 time
0  MAC000009  2012-09-20 10:00:00.0000000         0.119   2012-09-20 10:00:00
1  MAC000009  2012-09-20 10:30:00.0000000         0.168   2012-09-20 10:30:00
2  MAC000009  2012-09-20 11:00:00.0000000         0.293   2012-09-20 11:00:00
3  MAC000009  2012-09-20 11:30:00.0000000         0.747   2012-09-20 11:30:00
4  MAC000009  2012-09-20 12:00:00.0000000         0.186   2012-09-20 12:00:00


In [ ]:


df = pd.merge(df, house, on='LCLid', how='left')
df = pd.merge(df, hourly_weather, on='time',  how='left')
df['month']=[ int(v[5:7]) for v in df['tstp'].values]
df['dayofweek']=[datetime.strptime(v[:-6], '%Y-%m-%d %H:%M:%S.%f').weekday()  for v in df['tstp'].values]
df['isHoliday']=[v[:10] in hols  for v in df['tstp'].values]
df['halfhourofday'] = df.tstp.apply(lambda x: half_hour_of_day(x) )
df['date']=[ v[:10] for v in df['tstp'].values] 
df['ts_int'] = df.tstp.apply(lambda x: get_ts_int(x))
df = pd.merge(df, daily_weather, on='date', how='left')
df.sort_values(by='ts_int', ascending=True, inplace=True)
hcols =['h_visibility', 'h_windBearing', 'h_temperature','h_dewPoint', 'h_pressure', 'h_apparentTemperature', 'h_windSpeed',
   'h_precipType', 'h_icon', 'h_humidity', 'h_summary']
df[hcols]= df[hcols].ffill()
df[label_col] = pd.to_numeric(df[label_col], errors='coerce')
for c in categorical_cols:  # convert cat columns to ints
    df[c] = df[c].apply(lambda x: getvalue_for_column(c,x))